## Part A

**Step 1 – Create Storage Location in Snowflake**

Run the following SQL in **Snowflake Worksheet** to define the target database and schema.

```sql
CREATE DATABASE IF NOT EXISTS SAKILA_DW;
CREATE SCHEMA IF NOT EXISTS SAKILA_DW.CORE;
CREATE SCHEMA IF NOT EXISTS SAKILA_DW.PROCESSED;

**Step 2 – Open Cloud Shell & Install Required Libraries**

Open Google Cloud Shell and install dependencies for MySQL and Snowflake connectors.

```bash
pip install --upgrade "cloud-sql-python-connector[pymysql]" pandas sqlalchemy
pip install --upgrade "snowflake-connector-python[pandas]"

**Step 3 – Create Migration Script**

In Cloud Shell, create a Python file:

```bash
nano migrate_sakila_to_snowflake.py   (Raw Sakila Data)
nano new.py   (Processed Sakila Data)

✅ Save and exit with:
Ctrl + O → Enter → Ctrl + X

Raw Sakila Data

In [ ]:
# nano migrate_sakila_to_snowflake.py 

from google.cloud.sql.connector import Connector
import pymysql
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# === MySQL Connection (Cloud SQL) ===
INSTANCE_CONN_NAME = "dianachen1013-depa-sakila:us-central1:sakila-mysql"
MYSQL_USER = "root"
MYSQL_PASSWORD = "dianachen1013"
MYSQL_DB = "sakila"

# === Snowflake Connection ===
SF_USER = "DIANACHEN1013"
SF_PASSWORD = "Cdq20021013!!!"
SF_ACCOUNT = "ohishpe-ww65557"
SF_WAREHOUSE = "COMPUTE_WH"
SF_DATABASE = "SAKILA_DW"
SF_SCHEMA = "CORE"

# Connect to Snowflake
sf_conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)
print("✅ Connected to Snowflake")

# Connect to MySQL via Cloud SQL Connector
connector = Connector()
def get_mysql_conn():
    return connector.connect(
        INSTANCE_CONN_NAME,
        "pymysql",
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        db=MYSQL_DB
    )
mysql_conn = get_mysql_conn()
print("✅ Connected to MySQL via Cloud SQL Connector")

# Retrieve and migrate all tables
with mysql_conn.cursor() as cursor:
    cursor.execute("SHOW TABLES")
    tables = [row[0] for row in cursor.fetchall()]
print(f"📋 Found {len(tables)} tables: {tables}")

for tbl in tables:
    df = pd.read_sql(f"SELECT * FROM `{tbl}`", mysql_conn)
    df.columns = [c.upper() for c in df.columns]
    success, nchunks, nrows, _ = write_pandas(sf_conn, df, tbl.upper(),
                                             database=SF_DATABASE, schema=SF_SCHEMA, overwrite=True)
    print(f"✅ Migrated {tbl}: {nrows} rows")

mysql_conn.close()
connector.close()
sf_conn.close()
print("🎉 Migration finished successfully!")


Processed Sakila Data

In [ ]:
# new.py 

from google.cloud.sql.connector import Connector
import pymysql
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# === MySQL Connection (Cloud SQL) ===
INSTANCE_CONN_NAME = "dianachen1013-depa-sakila:us-central1:sakila-mysql"
MYSQL_USER = "root"
MYSQL_PASSWORD = "dianachen1013"
MYSQL_DB = "sakila_snowflake"

# === Snowflake Connection ===
SF_USER = "DIANACHEN1013"
SF_PASSWORD = "Cdq20021013!!!"
SF_ACCOUNT = "ohishpe-ww65557"
SF_WAREHOUSE = "COMPUTE_WH"
SF_DATABASE = "SAKILA_DW"
SF_SCHEMA = "PROCESSED"

# Connect to Snowflake
sf_conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)
print("✅ Connected to Snowflake")

# Connect to MySQL via Cloud SQL Connector
connector = Connector()
def get_mysql_conn():
    return connector.connect(
        INSTANCE_CONN_NAME,
        "pymysql",
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        db=MYSQL_DB
    )
mysql_conn = get_mysql_conn()
print("✅ Connected to MySQL via Cloud SQL Connector")

# Retrieve and migrate all tables
with mysql_conn.cursor() as cursor:
    cursor.execute("SHOW TABLES")
    tables = [row[0] for row in cursor.fetchall()]
print(f"📋 Found {len(tables)} tables: {tables}")

for tbl in tables:
    df = pd.read_sql(f"SELECT * FROM `{tbl}`", mysql_conn)
    df.columns = [c.upper() for c in df.columns]
    success, nchunks, nrows, _ = write_pandas(sf_conn, df, tbl.upper(),
                                             database=SF_DATABASE, schema=SF_SCHEMA, overwrite=True)
    print(f"✅ Migrated {tbl}: {nrows} rows")

mysql_conn.close()
connector.close()
sf_conn.close()
print("🎉 Migration finished successfully!")

**Step 4 – Run Migration Script in Cloud Shell**

Execute the migration with:

```bash
python3 migrate_sakila_to_snowflake.py   (Raw Sakila Data)
python3 new.py   (Processed Sakila Data)


Raw Sakila Data

![Snowflake Load](./GCP%20connect%20with%20Snowflake.png)

Processed Sakila Data

![Snowflake Load](./GCP%20processed%20connect%20with%20Snowflake.png)

**Step 5 – Check if raw sakila data and sakila_snowflake were successfully loaded to Snowflake**

Raw Sakila Data successfully loaded

![Rawdata Load](./Snowflake%20load%20sakila.png)

Processed Sakila Data successfully loaded

![Snowflake Load](./Snowflake%20load%20sakila_snowflake.png)
